In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('PracDF6').getOrCreate()

In [3]:
training=spark.read.option('header','true').csv('6_DF.csv',inferSchema=True)

In [4]:
c.show()

+-------+---+----------+------+
|   Name|Age|Experience|Salary|
+-------+---+----------+------+
|  Krish| 31|        10| 70000|
|Koustav| 28|         7| 80000|
|  Sunny| 32|        12|100000|
|  Harsh| 28|         8| 78000|
|  Suvam| 26|         5| 60000|
| Mainak| 28|         7| 80000|
+-------+---+----------+------+



In [5]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [10]:
##[Age,Experience]--->new feature--->independent feature
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["Age","Experience"],outputCol="Independent Features")

In [11]:
output=featureassembler.transform(training)

In [12]:
output.show()

+-------+---+----------+------+--------------------+
|   Name|Age|Experience|Salary|Independent Features|
+-------+---+----------+------+--------------------+
|  Krish| 31|        10| 70000|         [31.0,10.0]|
|Koustav| 28|         7| 80000|          [28.0,7.0]|
|  Sunny| 32|        12|100000|         [32.0,12.0]|
|  Harsh| 28|         8| 78000|          [28.0,8.0]|
|  Suvam| 26|         5| 60000|          [26.0,5.0]|
| Mainak| 28|         7| 80000|          [28.0,7.0]|
+-------+---+----------+------+--------------------+



In [13]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [15]:
finalized_data=output.select("Independent Features","Salary")

In [16]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 70000|
|          [28.0,7.0]| 80000|
|         [32.0,12.0]|100000|
|          [28.0,8.0]| 78000|
|          [26.0,5.0]| 60000|
|          [28.0,7.0]| 80000|
+--------------------+------+



In [17]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [18]:
###coefficients
regressor.coefficients

DenseVector([-10741.9355, 13838.7097])

In [19]:
### intercepts
regressor.intercept

272870.9677418024

In [20]:
### prdiction
pred_results=regressor.evaluate(test_data)

In [21]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [28.0,7.0]| 80000|68967.74193548539|
+--------------------+------+-----------------+



In [22]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(11032.258064514608, 121710718.00204761)